In [37]:
def input_matrix():
    n = int(input("정방행렬의 크기 n을 입력하세요: "))
    A = []
    for i in range(n):
        row = list(map(int, input().split()))
        A.append(row)
    return A

# 행렬 출력
def matrixout(mx, size):
    print("┌" + "   " * size + "┐")
    for i in range(size):
        print("|", end=" ")
        for j in range(size):
            print("%2.f" % mx[i][j], end=" ")
        print("|")
    print("└" + "   " * size + "┘")

# 전치 행렬
def transposeMatrix(m):
    return [[m[j][i] for j in range(len(m))] for i in range(len(m[0]))]

# 소행렬
def getMatrixMinor(m, i, j):
    return [row[:j] + row[j+1:] for row in (m[:i] + m[i+1:])]

# 행렬식 계산 (재귀 이용)
def getMatrixDeterminant(m):
    if len(m) == 1:
        return m[0][0]
    if len(m) == 2:
        return m[0][0]*m[1][1] - m[0][1]*m[1][0]

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1) ** c) * m[0][c] * getMatrixDeterminant(getMatrixMinor(m, 0, c))
    return determinant

# 역행렬 존재 여부 확인
def has_inverse(m):
    determinant = getMatrixDeterminant(m)
    if abs(determinant) < 1e-10:  # 부동소수점 오차 고려
        print("오류")
        print("해당 행렬의 행렬식은 0이므로 역행렬 계산은 불가능합니다!")
        return False
    return True

# 역행렬 계산
def getMatrixInverse(m):
    determinant = getMatrixDeterminant(m)

    if len(m) == 1:
        return [[1.0 / m[0][0]]]

    if len(m) == 2:
        return [[m[1][1] / determinant, -1 * m[0][1] / determinant],
                [-1 * m[1][0] / determinant, m[0][0] / determinant]]

    cofactors = []
    for r in range(len(m)):
        cofactorRow = []
        for c in range(len(m)):
            minor = getMatrixMinor(m, r, c)
            cofactorRow.append(((-1) ** (r + c)) * getMatrixDeterminant(minor))
        cofactors.append(cofactorRow)

    adjugate = transposeMatrix(cofactors)

    for r in range(len(adjugate)):
        for c in range(len(adjugate)):
            adjugate[r][c] = adjugate[r][c] / determinant

    return adjugate

# 가우스-조르단 소거법으로 역행렬 구하기
def getMatrixInverse_GJ(m):
    n = len(m)
    # 원본 행렬과 단위행렬을 합쳐 확장행렬 생성
    A = [row[:] for row in m]   # deepcopy
    I = [[int(i == j) for j in range(n)] for i in range(n)]

    for i in range(n):
        # pivot 선택 (대각원소)
        pivot = A[i][i]
        if abs(pivot) < 1e-10:
            raise ValueError("역행렬이 존재하지 않습니다. (pivot=0)")

        # pivot을 1로 만들기
        for j in range(n):
            A[i][j] /= pivot
            I[i][j] /= pivot

        # pivot 열의 다른 원소들을 0으로 만들기
        for k in range(n):
            if k != i:
                factor = A[k][i]
                for j in range(n):
                    A[k][j] -= factor * A[i][j]
                    I[k][j] -= factor * I[i][j]

    return I

def compare_results(inv1, inv2):
    n = len(inv1)
    m = len(inv1[0])
    for i in range(n):
        for j in range(m):
            if abs(inv1[i][j] - inv2[i][j]) > 1e-6:
                return False
    return True

def solve_linear_system(A, b):
    try:
        invA = getMatrixInverse_GJ(A)
        n = len(A)
        x = [0] * n
        for i in range(n):
            for j in range(n):
                x[i] += invA[i][j] * b[j]
        return x
    except Exception as e:
        print("연립방정식 풀이 오류:", e)
        return None



In [42]:
if __name__ == "__main__":
    A = input_matrix()

    print("\n입력 행렬 A:")
    matrixout(A, len(A))

    # 행렬식 방식 역행렬
    inv1 = None
    if has_inverse(A):
        print("\n행렬식 방식으로 구한 역행렬:")
        inv1 = getMatrixInverse(A)
        matrixout(inv1, len(A))

    # 가우스-조르단 방식 역행렬
    inv2 = None
    try:
        print("\n가우스-조르단 방식으로 구한 역행렬:")
        inv2 = getMatrixInverse_GJ(A)
        matrixout(inv2, len(A))
    except ValueError as e:
        print("가우스-조르단 방식 오류:", e)

    # 두 결과 비교
    if inv1 and inv2:
        if compare_results(inv1, inv2):
            print("\n두 방법의 결과가 동일합니다.")
        else:
            print("\n두 방법의 결과가 다릅니다.")

    print("\n--- 연립방정식 Ax=b 풀기 ---")
    b = list(map(int, input("b 벡터를 입력하세요 (공백 구분): ").split()))
    x = solve_linear_system(A, b)
    if x:
        print("\n해 x:")
        print([round(val, 4) for val in x])



정방행렬의 크기 n을 입력하세요: 5
3 2 1 4 2
1 1 1 5 2
8 2 3 1 2
4 2 3 1 1
4 2 4 7 4

입력 행렬 A:
┌               ┐
|  3  2  1  4  2 |
|  1  1  1  5  2 |
|  8  2  3  1  2 |
|  4  2  3  1  1 |
|  4  2  4  7  4 |
└               ┘

행렬식 방식으로 구한 역행렬:
┌               ┐
| -0  1  0 -0 -0 |
|  1 -1 -0  0  0 |
| -1  0 -0  0  0 |
| -1  1  0  0 -1 |
|  1 -3 -0 -1  1 |
└               ┘

가우스-조르단 방식으로 구한 역행렬:
┌               ┐
| -0  1  0  0 -0 |
|  1 -1 -0  0  0 |
| -1  0 -0  0  0 |
| -1  1  0  0 -1 |
|  1 -3 -0 -1  1 |
└               ┘

두 방법의 결과가 동일합니다.

--- 연립방정식 Ax=b 풀기 ---
b 벡터를 입력하세요 (공백 구분): 2 1 1 4 2

해 x:
[-0.3333, 2.0222, 0.8667, 0.3556, -1.6667]
